# Weather and Motor Vehicle Collisions

In [ ]:
import pandas as pd
from datetime import date
from dateutil.rrule import rrule, DAILY
from __future__ import division

pd.set_option('display.max_columns', None)

## Download weather data

In [ ]:
start_date = date(2012, 7, 1)
end_date = date(2016, 2, 29)

# data = pd.DataFrame()
frames = []
url_template = 'https://www.wunderground.com/history/airport/KNYC/%s/%s/%s/DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.zip=10001&reqdb.magic=4&reqdb.wmo=99999&format=1.csv'

month = ""

for dt in rrule(DAILY, dtstart=start_date, until=end_date):
    if (month != dt.strftime("%m")):
        month = dt.strftime("%m")
        print 'Downloading to memory: ' + dt.strftime("%Y-%m")    
    frames.append(pd.read_csv(url_template % (dt.strftime("%Y"),dt.strftime("%m"), dt.strftime("%d"))))

print "Saving data to csv..."
data = pd.concat(frames)
data.to_csv('weather_data_nyc.csv', sep=',')

## Cleaning the weather dataset
### Convert weather DateUTC to local time

In [ ]:
from datetime import datetime
from dateutil import tz

weather = pd.read_csv('datasets/weather_data_nyc_clean.csv')

def UTCtoActual(utcDate):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    
    utc = datetime.strptime(utcDate.DateUTC, '%m/%d/%Y %H:%M:%S')\
                  .replace(tzinfo=from_zone)\
                  .astimezone(to_zone)
    s = pd.Series([utc.year, utc.month, utc.day, utc.hour])
    s.columns = ['Year', 'Month', 'Day', 'Hour']
    return s
    
#weather['DateActual'] = weather.DateUTC.map()

In [ ]:
weather[['Year', 'Month', 'Day', 'Hour']] = weather.apply(UTCtoActual, axis=1)
weather.to_csv('datasets/weather_data_nyc_clean2.csv')

### Merge weather and NYPD MVC datasets

In [ ]:
incidents = pd.read_csv('datasets/NYPD_Motor_Vehicle_Collisions.csv')
weather = pd.read_csv('datasets/weather_data_nyc_clean2.csv')
incidents.head(1)

In [ ]:
features0 = ['Conditions', 'TemperatureC']
features = ['Conditions', 'Dew PointC', 'Events', \
            'Gust SpeedKm/h', 'Precipitationmm', \
            'TemperatureC', 'VisibilityKm', \
            'Wind SpeedKm/h', 'WindDirDegrees']

def lookup_weather2(year, month, day, hour):
    w = weather[(weather.Year == year) & (weather.Month == month) & (weather.Day == day) & (weather.Hour == hour)]
    return w

def lookup_weather(date, time):
    month = date.split('/')[0]
    day = date.split('/')[1]
    year = date.split('/')[2]
    hour = time.split(':')[0]
    return lookup_weather2(int(year), int(month), int(day), int(hour)).head(1)

def merge_weather(incident):
    date = incident.DATE
    time = incident.TIME
    
    w = lookup_weather(date, time)
    
    
    s = pd.Series([w['Conditions'], w['Dew PointC'], w['Events'], \
                   w['Gust SpeedKm/h'], w['Precipitationmm'], w['TemperatureC'], \
                   w['TemperatureC'], w['VisibilityKm'], w['Wind SpeedKm/h'], w['WindDirDegrees']])
    s.columns = features
    
    return s

#lookup_weather2(2016, 2, 14, 7)
lookup_weather('01/27/2015', '3:27')

In [ ]:
incidents[features] = incidents.apply(merge_weather, axis=1)
incidents.head()
incidents.to_csv('datasets/NYPD_Motor_Vehicle_Collisions_weather.csv', sep=',')

## Make some nice data analysis

In [ ]:
incidents = pd.read_csv('datasets/NYPD_Motor_Vehicle_Collisions.csv')
weather = pd.read_csv('datasets/weather_data_nyc_clean2.csv')

In [ ]:
weather.Conditions.unique()